# 京东数据爬取备份

In [1]:
from selenium import webdriver
import time
import random
import mysql.connector
import sys

In [2]:
WORD = '甜蜜蓝宝石葡萄'
TABLE = 'tmlbs'

# 用Selenium爬取数据并插入数据库

In [3]:
# TODO 1、创建浏览器
# 1.1 配置参数。
options = webdriver.ChromeOptions()
options.add_argument('blink-settings=imagesEnabled=false')  #不加载图片，提升运行速度
options.add_argument('--headless')  #无头模式
driver = webdriver.Chrome(options=options) 

In [4]:
# TODO 2、修改了浏览器的内部属性，跳过了登录的滑动验证
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument",
                        {"source": """Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"""})

{'identifier': '2'}

In [5]:
# TODO 3、执行浏览器操作
driver.get('https://www.jd.com/')  
driver.implicitly_wait(10)  # 智能化等待方法
# driver.maximize_window()  # 最大化

In [6]:
# 模拟搜索

driver.find_element("xpath", '//*[@id="key"]').send_keys(WORD)  
time.sleep(random.randint(1, 3))  
driver.find_element("xpath", '//*[@id="search"]/div/div[2]/button').click()
time.sleep(random.randint(1, 3)) 

In [7]:
# 链接数据库
cnx = mysql.connector.connect(host='localhost', user='root', password='Heruxi0000$', database='jd')
cursor = cnx.cursor()

In [8]:
data_length = 0
for page in range(0, 99):
    print(f'-----------------正在爬取第{page + 1}页-----------------',end=" ")
    # TODO 调用商品解析的函数
    divs = driver.find_elements("xpath", '//*[@id="J_goodsList"]/ul/li')  # 所有的div标签
    #print(len(divs))

    add_item = ("INSERT INTO {} " # 注意插入的表格是哪个
            "(grape_name, price, sale, store_name, tags, url) "
            "VALUES (%s, %s, %s, %s, %s, %s)".format(TABLE))
    item_data = []
    for div in divs:
        try: 
            test = div.find_element("xpath", './/div[@class="p-name p-name-type-2"]/a/em').text  # 商品名字
            price = div.find_element("xpath", './/div[@class="p-price"]').text  # 商品价格
            deal = div.find_element("xpath", './/div[@class="p-commit"]/strong/a').text  # 付款人数
            name = div.find_element("xpath", './/div[@class="p-shop"]/span/a').text  # 店铺名称
            tags_tree = div.find_elements("xpath", './/div[@class="p-icons"]/i') # 商品标签
            tags = ",".join([tag.text for tag in tags_tree])
            detail_url = div.find_element("xpath", './/div[@class="p-name p-name-type-2"]/a').get_attribute('href')  # 详情页地址
            # print(test, price, deal, name, detail_url).

            # 装载数据以便后期插入。
            item_data.append((test, price, deal, name, tags, detail_url))   
        except Exception as e:
            print(e)
            pass
    
    # 往数据库中插入数据
    try:
        cursor.executemany(add_item, item_data)
        data_length+=len(item_data)

    except mysql.connector.Error as e:
        print(e)
        cnx.rollback()
        cnx.close()
        sys.exit(1)

    # Make sure data is committed to the database
    cnx.commit()


    # 翻页
    driver.find_element("xpath", '//*[@id="J_bottomPage"]/span[1]/a[9]').click()
    time.sleep(random.randint(5, 15))    
    print("共插入{}条数据".format(data_length))
    
cursor.close()
cnx.close()


-----------------正在爬取第1页----------------- 共插入30条数据
-----------------正在爬取第2页----------------- 共插入60条数据
-----------------正在爬取第3页----------------- 共插入90条数据
-----------------正在爬取第4页----------------- 共插入120条数据
-----------------正在爬取第5页----------------- 共插入150条数据
-----------------正在爬取第6页----------------- 共插入180条数据
-----------------正在爬取第7页----------------- 共插入210条数据
-----------------正在爬取第8页----------------- 共插入240条数据
-----------------正在爬取第9页----------------- 共插入270条数据
-----------------正在爬取第10页----------------- 共插入300条数据
-----------------正在爬取第11页----------------- 共插入330条数据
-----------------正在爬取第12页----------------- 共插入360条数据
-----------------正在爬取第13页----------------- 共插入390条数据
-----------------正在爬取第14页----------------- 共插入420条数据
-----------------正在爬取第15页----------------- 共插入450条数据
-----------------正在爬取第16页----------------- 共插入480条数据
-----------------正在爬取第17页----------------- 共插入510条数据
-----------------正在爬取第18页----------------- 共插入540条数据
-----------------正在爬取第19页----------------- 共插入570条数据
-----

# 参考网站
1. Python使用pymysql模块的executemany()方法批量插入数据 提高Mysql数据库写入效率：https://blog.csdn.net/qq_41017902/article/details/107765724 